# Reference
This notebook aims to give a high-level overview of the Firefly Python client API and important methods. For **full explanation** of each method and its parameter, check [FireflyClient API reference](https://caltech-ipac.github.io/firefly_client/api/firefly_client.FireflyClient.html).

> Disclaimer: Since this is a reference notebook focussing on technical aspects, it's not going to be as scientifically interesting as the exercise notebooks that have real science use cases.

In [1]:
from firefly_client import FireflyClient
from astropy.utils.data import download_file

## Initialize Firefly Client
Firefly is a web application (with a URL) that can be controlled from Python with a `FireflyClient` object. 

To indicate which Firefly server to talk to from our Python client, we set the environment variable `FIREFLY_URL`. In this notebook, we use a public Firefly server: the IRSA Viewer (https://irsa.ipac.caltech.edu/irsaviewer). However, you can also run a local Firefly server via a [Firefly Docker image](https://hub.docker.com/r/ipac/firefly) and access it at `http://localhost:8080/firefly`.

There are two ways to initialize `FireflyClient` from Python, depending on whether you're running the notebook in JupyterLab or not:

### From Jupyter Lab
Assuming you have `jupyter-firefly-extensions` set up in your environment as explained [here](https://github.com/Caltech-IPAC/jupyter_firefly_extensions/blob/master/README.md), you can use `make_lab_client()` in JupyterLab, which will open the Firefly viewer in a new tab within the Lab.

In [2]:
fc = FireflyClient.make_lab_client()

### From Jupyter Notebook (or even a Python shell)
You can use `make_client()` in a Jupyter Notebook (or even a Python shell), which will open the Firefly viewer in a new web browser tab. `make_client()` also allows you to pass the URL directly (other than through environment variable) as the `url` parameter.

In [3]:
# fc = FireflyClient.make_client() # URL is taken from FIREFLY_URL env variable

# URL can be defined explicitly as a parameter
# fc = FireflyClient.make_client(url="https://irsa.ipac.caltech.edu/irsaviewer")

## [Optional] Reinitialize Firefly Client
To clean the state of Firefly server (aka remove all displayed components and start fresh), you can reinitialise it. This is specifically helpful if a Python connection with server is already open and re-running `make_lab_client` (or `make_client`) leads to stale state.

In [4]:
fc.reinit_viewer()

{'success': True}

## Show FITS Image in Firefly

### From a URL
You can specify url of any FITS image. Here we use cutout of a WISE all-sky image:

In [5]:
cutout_image_url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a/149/02206a149-w1-int-1b.fits?center=70,20&size=400pix'

In [6]:
cutout_image_plot_id = 'wise-cutout'
fc.show_fits(url=cutout_image_url,
             plot_id=cutout_image_plot_id, 
             title='WISE Cutout')

{'success': True}

### From a local file
You can specify path of any local FITS file. Here we download the full WISE all-sky image and use its file path:

In [7]:
full_image_url = 'http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a/149/02206a149-w1-int-1b.fits'
full_image_fpath = download_file(full_image_url, cache=True, timeout=120)
full_image_fpath

'/Users/jsinghal/.astropy/cache/download/url/8d0c853f180330d78cea76026057a827/contents'

Upload the local file to the Firefly server and then display it:

In [8]:
fval = fc.upload_file(full_image_fpath) # returns location on firefly server

# Pass fval to file_on_server parameter
full_image_plot_id = 'wise-fullimage'
fc.show_fits(file_on_server=fval,
             plot_id=full_image_plot_id, # using a different plot id to not overwrite previous plot
             title='WISE Full Image')

{'success': True}

## Modify the displayed image(s)
### Panning to a coordinate
To pan the full image to center on a cutout's center 70 +20 EQ-J2000:

In [9]:
fc.set_pan(plot_id=full_image_plot_id, # note we use the plot id we defined above
           x=70, y=20, coord='j2000')

{'success': True}

### Zooming
To zoom into the full image by a factor of 2:

In [10]:
fc.set_zoom(plot_id=full_image_plot_id, factor=2)

{'success': True}

### Aligning multiple images
To align both full image and cutout image by WCS and to lock the WCS matching:

In [11]:
fc.align_images(lock_match=True)

{'success': True}

### Changing the color stretch
Set the stretch for the full image based on IRAF zscale interval with a linear algorithm:

In [12]:
fc.set_stretch(plot_id=full_image_plot_id, stype='zscale', algorithm='linear')

{'success': True,
 'rv_string': '91,1.000000,91,1.000000,NaN,2.000000,44,25,600,120,0,NaN,1.000000'}

## Show Table/Catalogs in Firefly
### From a URL
You can specify URL of any table. Here we use a 2MASS catalog (using IRSA VO TAP):

In [13]:
table_url = "http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',70.0,20.0,0.1))=1"

In [14]:
table_id = '2mass-catalog'
fc.show_table(url=table_url,
              tbl_id=table_id,
              title='2MASS Catalog')

{'success': True}

### From a local file
You can specify path of any local table file. Here we download the table from above url and use its file path:

In [15]:
table_fpath = download_file(table_url, cache=True, timeout=120)
table_fpath

'/Users/jsinghal/.astropy/cache/download/url/c2a3ca8eac12ab6f9d563128eb1fa516/contents'

Similar to show_fits, upload the local file to the Firefly server and then display it:

In [16]:
fval = fc.upload_file(table_fpath) # returns location on firefly server

# Pass fval to file_on_server parameter
fc.show_table(file_on_server=fval,
              tbl_id=table_id, # using a similar table id to overwrite previous table
              title='2MASS Catalog')

{'success': True}

Note: Displaying a catalog table also caused Firefly to overlay the image with markers by default. The markers can be visible on both "wise-cutout" and "wise-fullimage" since they would include the same field of view. Also we see the active chart and coverage tab as bonus!

## Modify the displayed table


### Sorting by a column
You can sort the table by a column in ascending (ASC) or descending (DESC) order:

In [17]:
fc.sort_table_column(tbl_id=table_id,
                     column_name='j_m', sort_direction='ASC')

{'success': True}

### Filtering the rows
Filters can be specified as an SQL WHERE clause-like string with column names quoted:

In [18]:
fc.apply_table_filters(tbl_id=table_id, # note: we use the table id we defined above
                       filters='"j_m">15 and "j_m"<16 and "j_cmsig"<0.06')

{'success': True}

To remove the filters:

In [19]:
fc.apply_table_filters(tbl_id=table_id, filters='')

{'success': True}

## Plot Chart from the displayed table
We can use the dispalyed table data to plot columns on X and Y axis:

In [20]:
fc.show_xyplot(tbl_id=table_id, xCol='j_m', yCol='h_m-k_m')

{'success': True}

Alternatively, more generic method show_chart() can be used to create the same plot which accepts a lot of configuration parameters as [Plotly data structure](https://plotly.com/javascript/reference/index/):

In [21]:
trace0 = {'tbl_id': table_id, 'x': "tables::j_m", 'y': "tables::h_m-k_m", 
          'type' : 'scatter', 'mode': 'markers',
          'marker': dict(size=4, color='red', opacity=0.6)}
layout = {'title': '2MASS color-mag',
           'xaxis': dict(title='J'), 'yaxis': dict(title='H - K')}

status = fc.show_chart(data=[trace0], layout=layout)

## Plot Region overlays on the displayed image

[ds9-style regions](https://ds9.si.edu/doc/ref/region.html) can be overlaid on the image displayed by Firefly:

In [22]:
sources_box = "icrs;box 70d 20d 12' 12' 0d"
image_regions_id = 'wise-regions'
fc.overlay_region_layer(region_data=sources_box,
                        title='WISE all-sky Annotations', 
                        region_layer_id=image_regions_id)

{'success': True}

Add region data to an existing region layer:

In [23]:
ra, dec = 69.992614, 20.088488
point_source = f'icrs;point {ra}d {dec}d # point=cross 25 text={{Source A}}'
fc.add_region_data(region_data=point_source,
                   region_layer_id=image_regions_id) # note: we use same region_layer_id

{'success': True}

To remove region layer:

In [24]:
fc.delete_region_layer(image_regions_id)

{'success': True}

## Show HiPS image

In [25]:
hips_url = 'http://alasky.u-strasbg.fr/DSS/DSSColor'
hips_plot_id = 'hips-img-id'
target = '70;20;EQ_J2000'

fc.show_hips(plot_id=hips_plot_id, 
             hips_root_url=hips_url,
             Title='HiPS-DSS', 
             WorldPt=target)

{'success': True}

## Show 3-color image

### From URLs

In [26]:
# [W4, W2, W1] from WISE all-sky as [R, G, B] 
rgb_urls = ['https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w4-int-3.fits',
            'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w2-int-3.fits',
            'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p3am_cdd/07/0704/0704p196_ab41/0704p196_ab41-w1-int-3.fits']

In [27]:
three_color_params = [
    {
        'url': url,
        'Title': "3 color image"
    } for url in rgb_urls]

fc.show_fits_3color(three_color_params=three_color_params,
                    plot_id='3color-img')

{'success': True}

In [28]:
# Because we added more images, align and lock them by WCS again
fc.align_images(lock_match=True)

{'success': True}

### From local files

In [29]:
# rgb_fpaths = [download_file(img_url, cache=True, timeout=120) for img_url in rgb_urls]
# rgb_fpaths

In [30]:
# three_color_params = [
#     {
#         'file': fc.upload_file(fpath),
#         'Title': "3 color image"
#     } for fpath in rgb_fpaths]

# fc.show_fits_3color(three_color_params=three_color_params,
#                     plot_id='3color-img')

## Making the Most of this notebook

- Open the "Table of Contents" in the JupyterLab sidebar to quickly navigate to specific sections needed for solving exercises.  
- To learn more about a method demonstrated here, refer to the "Methods Summary" table in the [FireflyClient API reference](https://caltech-ipac.github.io/firefly_client/api/firefly_client.FireflyClient.html). Click on the method to explore its definition and parameters in detail.